In [1]:
%load_ext autoreload
%autoreload 2
import os 
import glob
import numpy as np
# import matplotlib.pyplot as plt
import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir,'..')))
from src import *

In [2]:
# ColorChecker spec lib
spec_srcpath = os.path.join(current_dir,'ColorChecker/spectres')
srcflist = np.array(glob.glob(os.path.join(spec_srcpath,'*.csv')))
splib_names = [f[len(spec_srcpath)+1:-len('.csv')] for f in srcflist]
print('Total number of ECOSTRESS spectra',len(splib_names))

Total number of ECOSTRESS spectra 24


In [3]:
# D65 spec
ds_d65 = read_csv(os.path.join(current_dir,'CIE','CIE_std_illum_D65.csv'))
wls = np.array(ds_d65['WL'])

def select_d65(ds_d65, srcflist):
    ds_color = read_csv(os.path.join(srcflist[0]))
    wl = []
    d65 = []
    for i in range(len(ds_color['wl'])):
        for j in range(len(ds_d65['WL'])):
            if ds_color['wl'][i] == ds_d65['WL'][j]:
                d65.append(ds_d65['D65'][j])
                wl.append(ds_d65['WL'][j])
    return d65, wl

d65, wl = select_d65(ds_d65, srcflist)

In [4]:
# Sentinel-2 SRF
ds_srf_s2a = read_csv(os.path.join(current_dir,'SRF','S2A-SRF.csv'))
ds_srf_s2b = read_csv(os.path.join(current_dir,'SRF','S2B-SRF.csv'))
s2_bnames = ['B2','B3','B4']
sensor_names = ['S2A','S2B']
ds_srf_s2 = [ds_srf_s2a,ds_srf_s2b]

def select_sentinel(ds_srf_s2a, srcflist):
    ds_color = read_csv(os.path.join(srcflist[0]))
    B1 = []
    B2 = []
    B3 = []
    for i in range(len(ds_color['wl'])):
        for j in range(len(ds_srf_s2a['WL'])):
            if ds_color['wl'][i] == ds_srf_s2a['WL'][j]:
                B1.append(ds_srf_s2a['B2'][j])
                B2.append(ds_srf_s2a['B3'][j])
                B3.append(ds_srf_s2a['B4'][j])
    return B1,B2,B3

#sentinel 2A
B2, B3, B4 = select_sentinel(ds_srf_s2a, srcflist)
ds_srf_s2a['WL'] = wl
ds_srf_s2a['B2'] = B2
ds_srf_s2a['B3'] = B3
ds_srf_s2a['B4'] = B4
#print(ds_srf_s2a)

#sentinel 2B
B2, B3, B4 = select_sentinel(ds_srf_s2b, srcflist)
ds_srf_s2b['WL'] = wl
ds_srf_s2b['B2'] = B2
ds_srf_s2b['B3'] = B3
ds_srf_s2b['B4'] = B4
#print(ds_srf_s2b)

In [5]:
# load xyz
ds_xyz = read_csv(os.path.join(current_dir,'CIE','cie_1931_2deg_xyz_1nm.csv'))

def select_cie(ds_xyz, srcflist):
    ds_color = read_csv(os.path.join(srcflist[0]))
    X = []
    Y = []
    Z = []
    
    for i in range(len(ds_color['wl'])):
        for j in range(len(ds_xyz['WL'])):
            if ds_color['wl'][i] == ds_xyz['WL'][j]:
                X.append(ds_xyz['X'][j])
                Y.append(ds_xyz['Y'][j])
                Z.append(ds_xyz['Z'][j])
    return X,Y,Z

X,Y,Z = select_cie(ds_xyz, srcflist)
ds_xyz['WL'] = wl
ds_xyz['X'] = X
ds_xyz['Y'] = Y
ds_xyz['Z'] = Z
#print(ds_xyz)

In [6]:
def band_resp(spec,srf):
    val = np.sum(spec*srf)/np.sum(srf)
    return val

In [7]:
# Calculate the constants of D65 of three S2 bands
def simu_sentinel2_d65(ds_srf,s2_bnames,d65,dstfile):
    mydict = {}
    mydict['BNAME'] = s2_bnames
    vals = [] 
    for i in range(len(s2_bnames)):
        bname = s2_bnames[i]
        srf = np.array(ds_srf[bname])
        val = band_resp(d65,srf)
        vals.append(val)
    mydict['D65'] = np.array(vals)
    write_csv(dstfile,mydict)
    
for i in range(len(ds_srf_s2)):
    simu_sentinel2_d65(ds_srf_s2[i],s2_bnames,d65,os.path.join(current_dir,'D65_color'+sensor_names[i]+'.csv'))

In [8]:
# Sentinel-2 A/B simulation
def simu_sentinel2_refl(ds_srf,s2_bnames,d65,srcflist,splib_names,dstfile):
    mydict = {}
    mydict['NAME'] = splib_names
    for i in range(len(s2_bnames)):
        bname = s2_bnames[i]
        srf = np.array(ds_srf[bname])
        vals = []    
        for j in range(len(srcflist)):
            srcfile = srcflist[j]
            ds_spec = read_csv(srcfile)
            spec = np.array(ds_spec['value'])
            # rho
            val = band_resp(spec,d65*srf)
            vals.append(val)
        mydict[bname] = np.array(vals)
    write_csv(dstfile,mydict)

def simu_sentinel2_rad(ds_srf,s2_bnames,d65,srcflist,splib_names,dstfile):
    mydict = {}
    mydict['NAME'] = splib_names
    for i in range(len(s2_bnames)):
        bname = s2_bnames[i]
        srf = np.array(ds_srf[bname])
        vals = []    
        for j in range(len(srcflist)):
            srcfile = srcflist[j]
            ds_spec = read_csv(srcfile)
            spec = np.array(ds_spec['value'])
            # rad
            val = band_resp(spec*d65,srf)
            vals.append(val)
        mydict[bname] = np.array(vals)
    write_csv(dstfile,mydict)
    

for i in range(len(ds_srf_s2)):
    simu_sentinel2_refl(ds_srf_s2[i],s2_bnames,np.array(d65),srcflist,splib_names,os.path.join(current_dir,'rho_ColorCheck_'+sensor_names[i]+'.csv'))
    simu_sentinel2_rad(ds_srf_s2[i],s2_bnames,np.array(d65),srcflist,splib_names,os.path.join(current_dir,'rad_ColorCheck_'+sensor_names[i]+'.csv'))


In [9]:
# XYZ
def simu_xyz(ds_xyz,d65,srcflist,splib_names,dstfile):
    my_dict = {}
    my_dict['NAME'] = splib_names
    xyz_names = list(ds_xyz.keys())[1:]
    # the normalized constant N
    srf = np.array(ds_xyz['Y'])
    N = np.sum(d65*srf)
    for i in range(len(xyz_names)):
        name = xyz_names[i]
        srf = np.array(ds_xyz[name])
        vals = []    
        for j in range(len(srcflist)):
            srcfile = srcflist[j]
            ds_spec = read_csv(srcfile)
            spec = np.array(ds_spec['value'])
            val = np.sum(spec*d65*srf)/N          
            vals.append(val)
        my_dict[name] = np.array(vals)
    write_csv(dstfile,my_dict)
 
simu_xyz(ds_xyz,np.array(d65),srcflist,splib_names,os.path.join(current_dir,'xyz_ColorCheck.csv'))
